In [1]:
import ruamel.yaml as yl
import pandas as pd
import numpy as np

In [2]:
#in_file is the address for data model
in_file = "..."
yaml = yl.YAML(typ='safe')
with open(in_file, encoding='utf-8-sig') as file:
    data_model = yaml.load(file)

In [3]:
data_model

{'Nodes': {'institution': {'Props': ['INSTITUTION_NAME']},
  'study_subject': {'Props': ['SUBJECT_ID',
    'KARNOFSKY',
    'NEURO_EXAM_SCORE',
    'DISEASE_EVAL_MRI',
    'STEROID_DOSE_STATUS',
    'ANTI_CONVULSANT_STATUS']},
  'demographic_data': {'Props': ['RACE', 'AGE_RANGE', 'GENDER']},
  'prior_therapeutic_procedure': {'Props': ['PRIOR_THERAPY_RADIATION_TYPE',
    'PRIOR_THERAPY_CHEMO_AGENT_NAME',
    'PRIOR_THERAPY_PROCEDURE',
    'PRIOR_THERAPY_SURGERY_TUMOR_HISTOLOGY',
    'PRIOR_THERAPY_SURGERY_OUTCOME',
    'PRIOR_THERAPY_SURGERY_OUTCOME_2',
    'PT_NAME2',
    'PRIOR_THERAPY_SURGERY_OUTCOME_1',
    'PT_NAME1']},
  'on_therapeutic_procedure': {'Props': ['ON_THERAPY_RADIATION_SITE',
    'ON_THERAPY_RADIATION_TYPE',
    'ON_THERAPY_CHEMO_AGENT_NAME',
    'ON_THERAPY_SURGERY_TITLE',
    'ON_THERAPY_SURGERY_HIST_DIAG',
    'ON_THERAPY_SURGERY_OUTCOME']},
  'diagnosis': {'Props': ['DISEASE_TYPE', 'WHO_GRADE']},
  'outcome': {'Props': ['EVENT_OS', 'OVERALL_SURVIVAL_MONTHS']}},
 'R

In [4]:
data_model["Nodes"].keys()

dict_keys(['institution', 'study_subject', 'demographic_data', 'prior_therapeutic_procedure', 'on_therapeutic_procedure', 'diagnosis', 'outcome'])

In [5]:
data_model["Nodes"]["study_subject"]['Props']

['SUBJECT_ID',
 'KARNOFSKY',
 'NEURO_EXAM_SCORE',
 'DISEASE_EVAL_MRI',
 'STEROID_DOSE_STATUS',
 'ANTI_CONVULSANT_STATUS']

In [6]:
#read in the data(tsv)
data = pd.read_csv('...', sep='\t')

In [7]:
data.columns

Index(['SUBJECT_ID', 'PARENT_ID', 'TYPE', 'TIMEPOINT', 'AGE_RANGE', 'GENDER',
       'EVENT_OS', 'OVERALL_SURVIVAL_MONTHS', 'DISEASE_TYPE', 'WHO_GRADE',
       'RACE', 'INSTITUTION_NAME', 'KARNOFSKY', 'NEURO_EXAM_SCORE',
       'DISEASE_EVAL_MRI', 'STEROID_DOSE_STATUS', 'ANTI_CONVULSANT_STATUS',
       'PRIOR_THERAPY_RADIATION_TYPE', 'PRIOR_THERAPY_CHEMO_AGENT_NAME',
       'PRIOR_THERAPY_SURGERY_TUMOR_HISTOLOGY', 'PRIOR_THERAPY_PROCEDURE',
       'PRIOR_THERAPY_SURGERY_OUTCOME', 'ON_THERAPY_RADIATION_SITE',
       'ON_THERAPY_RADIATION_TYPE', 'ON_THERAPY_CHEMO_AGENT_NAME',
       'ON_THERAPY_SURGERY_HIST_DIAG', 'ON_THERAPY_SURGERY_TITLE',
       'ON_THERAPY_SURGERY_OUTCOME', 'PRIOR_THERAPY_SURGERY_OUTCOME_2',
       'PT_NAME2', 'PRIOR_THERAPY_SURGERY_OUTCOME_1', 'PT_NAME1', 'GENE_EXP',
       'COPY_NUMBER'],
      dtype='object')

In [8]:
study_subject = data_model["Nodes"]["study_subject"]['Props']

In [9]:
A = data.loc[:, study_subject]

In [10]:
colname = 'study_subject' + '_id'
A.insert(0, colname, A.index + 1)

In [11]:
A[colname] =  colname + '_' + A[colname].astype(str)

In [12]:
A.insert(0, 'type', 'study_subject')

In [13]:
A.head()

,type,study_subject_id,SUBJECT_ID,KARNOFSKY,NEURO_EXAM_SCORE,DISEASE_EVAL_MRI,STEROID_DOSE_STATUS,ANTI_CONVULSANT_STATUS
0,study_subject,study_subject_id_1,860772,NaN,NaN,NaN,NaN,NaN
1,study_subject,study_subject_id_2,3089254,NaN,NaN,NaN,NaN,NaN
2,study_subject,study_subject_id_3,3495697,NaN,NaN,NaN,NaN,NaN
3,study_subject,study_subject_id_4,3497265,NaN,NaN,NaN,NaN,NaN
4,study_subject,study_subject_id_5,3504050,NaN,NaN,NaN,NaN,NaN


In [14]:
# use if else to search Src and Dst to determine if a parent ID column need to be add  

In [15]:
for x in data_model["Relationships"].keys():
    for a in data_model["Relationships"][x]['Ends']:
        b = a.get('Src')
        print(b)

demographic_data
outcome
diagnosis
institution
on_therapeutic_procedure
prior_therapeutic_procedure


In [16]:
#table = data.copy()
for i in data_model["Nodes"].keys():

    colnames = data_model["Nodes"][i]['Props']
    table = data.loc[:, colnames]
    IDcol = i + '_id'
    table.insert(0, IDcol, table.index + 1)
    table[IDcol] =  IDcol + '_' + table[IDcol].astype(str)
    for x in data_model["Relationships"].keys():
        for a in data_model["Relationships"][x]['Ends']:
            b = a.get('Src')
            if b == i:
                parentname = a.get('Dst')
                Parcol = parentname + '.' + parentname + '_id'
                table.insert(0, Parcol, table.index + 1)
                table[Parcol] =  parentname + '_id_' + table[Parcol].astype(str)
    table.insert(0, 'type', i)
    filename = i + '_file.txt'
    table.to_csv('./output/' + filename, sep = '\t', index = False)
#    print(table.head())

In [17]:
table.head()

,type,study_subject.study_subject_id,outcome_id,EVENT_OS,OVERALL_SURVIVAL_MONTHS
0,outcome,study_subject_id_1,outcome_id_1,NaN,NaN
1,outcome,study_subject_id_2,outcome_id_2,NaN,NaN
2,outcome,study_subject_id_3,outcome_id_3,NaN,NaN
3,outcome,study_subject_id_4,outcome_id_4,NaN,NaN
4,outcome,study_subject_id_5,outcome_id_5,NaN,NaN
